# ML Trader

In [ ]:
from utils.packages import *
from utils.time_tick import *
from utils.variables import *
from utils.i_o import *
from utils.dir_slope import *
from utils.engine import *
from utils.loops import *
from utils.order import *
from config.access_keys import *
warnings.filterwarnings('ignore')

#--------------------------------------------------------------------------------------------------------------------------
# Run

if data['run_type'] == 'single':
    data = run_engine(data)        

elif data['run_type'] == 'loop':
    while data["run_flg"] ==  True:
        try:        
            data = run_engine(data)        
        
        except KeyboardInterrupt:
            print("Run manually stopped")
            ts = dt.datetime.now()
            err_msg = 'KeyboardInterrupt'
            logging.error(f'--- Timestamp-{ts}, Error-{err_msg}')
            break           
        
        except Exception as err_msg:
            data['error_count'] = data['error_count'] + 1
            ts = dt.datetime.now()
            logging.error(f'--- Timestamp-{ts}, Error-{err_msg}')
#==========================================================================================================================            

data = check_for_open_orders(data)
data = check_for_open_orders(data)
data = check_for_open_orders(data)

In [ ]:
#--------------------------------------------------------------------------------------------------------------------------
# ### Inputs and Parameters   
    
logging.basicConfig(filename='traderrun.log', level=logging.ERROR)

data['access_token'] = oanda_demo_2['token']
data['accountID'] = oanda_demo_2['account_id']
data['params'] = {'instruments': data['instrument']}

data['api'] = API(access_token = data['access_token'])
data['request_stream_data'] = pricing.PricingStream(accountID=data['accountID'], params=data['params'])
data['response_stream'] = data['api'].request(data['request_stream_data'])
data["start_ts"] = dt.datetime.strftime(dt.datetime.now(), '%Y-%m-%d-%H-%M')

data = calculate_purchasable_EURUSD_units(data)
# ------------------------------------------------------------------------------------

In [ ]:
data['df'] = pd.DataFrame()

data['df']['DateTime_frmt'] = np.nan
data['df']['bid']  = np.nan
data['df']['ask'] = np.nan
data['df']['tick'] = np.nan
data['df']['lema'] = np.nan
data['df']['h'] = np.nan
data['df']['l'] = np.nan
# data['df']['h_avg'] = np.nan
# data['df']['l_avg'] = np.nan
data['df']['h_gap'] = np.nan
data['df']['l_gap'] = np.nan
data['df']['h_lema'] = np.nan
data['df']['l_lema'] = np.nan

In [ ]:
for data['iter'], data['resp'] in enumerate(data['response_stream']):    

    data["start_ts"] = dt.datetime.strftime(dt.datetime.now(), '%Y-%m-%d-%H-%M')

    # Heartbeat check ---------------------------------------------
    if data['resp']['type'] == 'HEARTBEAT': # Heart beat response to keep the data['api'] connection alive (Avoid timeout)
        continue

    # Get time ---------------------------------------------
    data = get_date_time(data)


    # Time diff check ---------------------------------------------
    if data['time_diff'] > data['max_time_diff']:        
        raise ValueError(f'Time difference is above limit : {data["time_diff"]}')

    # Get Data ---------------------------------------------
    data = get_prices(data)

#     print_report(data)
    

    # lema before after loops --------------------------------
    if len(data['tick_list']) < data['lema_len']:
        data = before_lema(data)        
        display.clear_output(wait = True) 
        print(data['iter'])
        continue        

    if len(data['tick_list']) == data['lema_len']:
        data = after_lema(data)                  
    # ----------------------------------------------------------
     
    data = capture_in_df(data)
    
    display.clear_output(wait = True)    
#     print(data['df'][['tick', 'lema', 'h', 'l', 'h_gap', 'l_gap', 'h_lema', 'l_lema']])
    plot_plotly_graph(data)
    
#     print(data['df'])
#     data['df']['h_l_gap']       = data['tot_ts_2']
#     data['df']['h_lema']        = data['tot_ts_2']
#     data['df']['l_lema']        = data['tot_ts_2']    

#     data = get_position(data)

#     if not data['position']:
#         continue

#     # Get Dirs --------------------------------
#     if len(data['dir_list']) < 2:
#         data['dir_list'].append(data['position'])   
#         continue

#     elif len(data['dir_list']) == 2:
#         data = get_cross_dir(data)
#     # ----------------------------------------------------------  


#     data = check_for_open_orders(data)
#     data = get_candle_size(data)
#     data = set_take_profit(data)
#     data = angle_close(data)

#     data = check_for_open_orders(data)
#     data = reverse_order(data)

#     data = check_for_open_orders(data)
#     data = make_order(data)    

In [ ]:
data['df'].tail()

In [ ]:
data['df']['h'][-10:].mean() - data['df']['lema']

In [ ]:
sys.exit()

In [ ]:
data['df'][-10:]

In [ ]:
params = {"count": 1, "granularity": 'M5'}
candle_size_data = instruments.InstrumentsCandles(instrument=data['curr_conv_pair'], params=params)
data['candle_size_info'] = data["api"].request(candle_size_data)
data['candle_size_info']
# data['convertion_rate'] = float([candle['mid']['c'] for candle in data['candle_size_info']['candles']][0])